### 네이버 뉴스 정보 가져오기

### 학습 내용 
 * 네이버 뉴스 여러개 단어 검색
 * 페이지별 언론사, 뉴스기사, URL

### 01. 웹 브라우저 띄우기

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

### 네이버 뉴스 검색

In [2]:
driver = webdriver.Chrome('chromedriver_90')

url = 'https://search.naver.com/search.naver?query=apple&where=news'
driver.get(url)

import time
time.sleep(3)  # 홈페이지 로딩 시간 3초

In [3]:

keyword_multi = ['2021년 봄 패션 트렌드', 
                 '2021년 여름 패션 트렌드', 
                 '코로나 검사', 
                 '코로나 확진자', 
                 '백신 현황']

pages = 30

In [4]:
### 뉴스 검색 : //*[@id="nx_query"]
### 검색 버튼 : //*[@id="lnb.searchForm"]/fieldset/button

In [5]:
from datetime import datetime

now = datetime.now()
file_make_time = "%04d_%02d%02d_%02d" % (now.year, now.month, now.day, now.hour)
print(now.day - 1)
now_day = now.day
now_hour = now.hour
file_make_time

1


'2021_0702_16'

In [6]:
import re
from datetime import timedelta

In [7]:
def date_cal(now_time):
    now = datetime.now()

    # 시간이 포함되어 있고,
    chk_time = now_time.find("시간")
    chk_day = now_time.find("일")
    if chk_time>=0: 
        hour = int( re.findall("\d+", now_time)[0] )
        
        # 현재 시간 - 적힌 시간이 음수가 나오면 현재 날짜에 -1을 해 준다. 
        cal_hour = now.hour - hour
        # print(cal_hour)
        if cal_hour < 0:
            news_date = timedelta(days = 1)
            cal_now = now - news_date
            #print("현재 날짜 :",  cal_now )
        else:
            news_date = timedelta(days = 0)
            cal_now = now - news_date
            #print("현재 날짜 :",  cal_now )
            
        cal_date_time = "%04d-%02d-%02d" % (cal_now.year, cal_now.month, cal_now.day)
        print("뉴스 날짜 : ", cal_date_time)
    
    # 일이 포함되어 있고,
    elif chk_day>=0: 
        news_day = int( re.findall("\d+", now_time)[0] )
        # print(news_day)
        
        # 현재 날짜 -1일전을 계산한다.
        news_date = timedelta(days = news_day)
        cal_now = now - news_date
        
        cal_date_time = "%04d-%02d-%02d" % (cal_now.year, cal_now.month, cal_now.day)
        # print("뉴스 날짜 : ", cal_date_time)
    else:
        cal_date_time = now_time
    return cal_date_time    

In [8]:
# date_cal("3일 전")

In [9]:
def keyword_news_n(keyword1, now_time):
    now_page_num = 0
    all_company1 = []
    all_article1 = []
    all_link1 = []
    all_date1 = []
    
    for i in range(1,pages,1):
        # 페이지 부분 정보 획득
        sel_num = driver.find_element_by_class_name("sc_page_inner")
        sel_all = sel_num.find_elements_by_class_name("btn")
        
        # 화면에 보이는 페이지 확인
        print("현재 페이지 : ", now_page_num+1, "키워드 : ", keyword1, "작성시간: ", now_time)
        len_all = len(sel_all)
        
        # 페이지 번호 확인 후, 해당되는 버튼 클릭
        for i in range(1, len_all+1 ):
            tmp_pre = '//*[@id="main_pack"]/div[2]/div/div/a[' + str(i) + ']'
            sel_num = driver.find_element_by_xpath(tmp_pre)
            sel_page_num = int(sel_num.text)
            if sel_page_num == (now_page_num+1):
                # 정보를 가져와서 클릭
                # 현재 페이지 입력
                sel_num.click()
                time.sleep(1)
                now_page_num += 1 
                print(now_page_num)
                break

        page = driver.page_source
        soup = BeautifulSoup(page, 'html.parser')
        all_ul = soup.find("ul", class_="list_news").find_all("li", class_="bx")

        for one in all_ul:
            # 언론사
            com = one.find("a", class_="info press").text
            com_a = com.replace("언론사 선정", "")

            # print(one)
            tmp = one.find("a", class_="news_tit")

            article_n = tmp.text
            tmp_link = tmp.attrs['href']
            
            # 날짜
            date_one = one.find('div', class_='info_group').find("span", class_="info").text
            now_dates = date_cal(date_one)  # 날짜 계산
            
            print(article_n, com_a, now_dates)
            print("url : ", tmp_link)

            all_company1.append(com_a)
            all_article1.append(article_n)
            all_link1.append(tmp_link)
            all_date1.append(now_dates)
            
        print(len(all_article))
    return all_company1, all_article1, all_link1, all_date1

In [10]:
import pandas as pd
from datetime import datetime
import os

### 폴더 생성

In [11]:
now = datetime.now()
file_make_time = "%04d_%02d%02d_%02d" % (now.year, now.month, now.day, now.hour)

try:
    if not os.path.exists(file_make_time):
        os.makedirs(file_make_time)
        print("folder creating ok")
except OSError:
    print("Error : Creating directory ", file_make_time)

folder creating ok


In [ ]:
now = datetime.now()
file_make_time = "%04d_%02d%02d_%02d" % (now.year, now.month, now.day, now.hour)

all_company = []
all_article = []
all_link = []
all_date = []

for one in keyword_multi:
    url = 'https://search.naver.com/search.naver?query=' + one + '&where=news'
    driver.get(url)
    time.sleep(3)
    
    sel_search = driver.find_element_by_xpath('//*[@id="nx_query"]') 
    sel_btn = driver.find_element_by_xpath('//*[@id="nx_search_form"]/fieldset/button') 
    
    sel_search.clear()
    sel_search.send_keys(one)
    sel_btn.click()
    
    all_company, all_article, all_link, all_date = keyword_news_n(one, 
                                                                  file_make_time)
    
    dict_dat = {"언론사": all_company, "기사명" : all_article, 
                "날짜":all_date, "URL":all_link}
    
    dat = pd.DataFrame(dict_dat)
    dat['키워드'] = one
    dat.columns = ['키워드', "언론사", "기사명", "날짜", "URL"]
    
    fname_csv = file_make_time + "/" + one + "_" + file_make_time +".csv"
    fname_xlsx = file_make_time + "/" + one + "_" + file_make_time +".xlsx"
    dat.to_csv(fname_csv, index=False)
    dat.to_excel(fname_xlsx, index=False)